# 1. Setting HuggingFace and importing modules

In [7]:
import sys
import json
import os

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install huggingface_hub

In [ ]:
#!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
#from huggingface_hub import login

# Replace 'your_new_huggingface_token' with the token you just generated
#login(token='LOGIN_TOKEN')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 2. Token Classification task

In [3]:
pip install transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=36c3b1cc7ec0e4f3c324a2f6b2e69f78b1848b6f3d490670386b10d80b9e83ef
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [9]:
!pip install datasets==2.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1


In [2]:
import transformers

print(transformers.__version__)

4.44.2


In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

# Model: MarIA

In [4]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "PlanTL-GOB-ES/roberta-large-bne"
batch_size = 16

## Another way of uploading a dataset

In [1]:
#from datasets import load_dataset, DatasetDict, load_metric, Dataset
from datasets import Dataset, load_dataset, load_metric

In [10]:
data_files = {"train": "drive/MyDrive/data/TRAIN.json", "dev": "drive/MyDrive/data/DEV.json", "test": "drive/MyDrive/data/TEST.json"}

In [11]:
datasets = load_dataset('json', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [12]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2308
    })
    dev: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1714
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2097
    })
})

In [13]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

## Ver etiquetas del dataset

In [14]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(datasets["train"])

,tokens,ner_tags
0,"[•, cualquier, otra, afección, médica, grave, (, equivalente, a, CTCAE, grado, 3, o, superior, ), que, aumente, sustancialmente, el, riesgo, de, sufrir, AA]","[O, O, O, B-CW, I-CW, O, O, O, O, B-CW, O, O, O, O, O, O, O, O, O, O, O, O, B-CW]"
1,"[2, .]","[O, O]"
2,"[Título, público, :, Estudio, para, evaluar, la, eficacia, ,, la, seguridad, ,, la, farmacocinética, y, la, farmacodinámica, de, ION363, en, pacientes, con, esclerosis, lateral, amiotrófica, con, mutaciones, del, gen, fusionado, en, sarcoma, (, ELA-FUS, )]","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-CW, O, O, B-CW, O, B-CW, O, O, O, B-CW, I-CW, I-CW, O, B-CW, O, B-CW, I-CW, I-CW, I-CW, O, B-CW, O]"
3,"[Hipotiroidismo, no, controlado, :, definido, como, estar, en, el, inicio, de, una, terapia, de, sustitución, de, la, hormona, tiroidea, o, en, proceso, de, ajuste, de, la, dosis, de, la, terapia, de, sustitución, en, las, 12, semanas, anteriores, al, cribado, .]","[B-CW, O, O, O, O, O, O, O, O, O, O, O, B-CW, I-CW, I-CW, I-CW, I-CW, I-CW, I-CW, O, O, O, O, O, O, O, O, O, O, B-CW, I-CW, I-CW, O, O, O, O, O, O, B-CW, O]"
4,"[Indicación, pública, :, osteoartritis, de, rodilla]","[B-CW, O, O, B-CW, I-CW, I-CW]"
5,"[El, participante, tiene, un, diagnóstico, clínico, confirmado, de, enfermedad, hepática, crónica, (, alanina, aminotransferasa, [, ALT, ], o, aspartato, aminotransferasa, [, AST, ], superior, a, 3, veces, el, límite, superior, de, la, normalidad, ), en, la, visita, de, selección, .]","[O, O, O, O, B-CW, I-CW, O, O, B-CW, I-CW, O, O, B-CW, I-CW, O, B-CW, O, O, B-CW, I-CW, O, B-CW, O, O, O, O, O, O, B-CW, I-CW, I-CW, I-CW, I-CW, O, O, O, O, O, O, O]"
6,"[-, Adecuada, función, orgánica, :, A., Función, renal, adecuada, ,, definida, como, *, TFGe, calculada, (, según, la, fórmula, de, Schwartz, ,, Apéndice, III, ), o, TFG, por, radioisótopos, ≥60, ml, /, min/1,73, m2, *, Creatinina, sérica, en, función, de, la, edad, /, sexo, .]","[O, O, B-CW, I-CW, O, O, B-CW, I-CW, O, O, O, O, O, B-CW, O, O, O, O, B-CW, I-CW, I-CW, O, O, O, O, O, B-CW, O, B-CW, O, O, O, O, O, O, B-CW, I-CW, O, O, O, O, O, O, O, O]"
7,"[11, .]","[O, O]"
8,"[Pacientes, que, no, tienen, respuesta, suficiente, al, tratamiento, habitual, de, combinación, de, baclofeno, y, otro, fármaco, antiespástico, :, tizanidina, ,, diazepam, ,, clonazepam, ,, gabapentinoides, (, gabapentina, ,, pregabalina, ), .]","[O, O, O, O, O, O, O, O, O, O, O, O, B-CW, O, O, B-CW, I-CW, O, B-CW, O, B-CW, O, B-CW, O, B-CW, O, B-CW, O, B-CW, O, O]"
9,"[1, .]","[O, O]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/858k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/516k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [17]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [18]:
example = datasets["train"][10]
print(example["tokens"])

['a.', 'La', 'cohorte', 'A', 'deben', 'tener', 'entre', '12', 'y', '65', 'años', ',', 'ambos', 'inclusive', ',', 'en', 'el', 'momento', 'de', 'obtener', 'el', 'consentimiento', 'informado', ',', 'así', 'como', 'signos', 'o', 'síntomas', 'compatibles', 'con', 'ELA', 'en', 'opinión', 'del', 'investigador', 'y', ',', 'en', 'caso', 'de', 'tener', 'entre', '30', 'y', '65', 'años', ',', 'ambos', 'inclusive', ',', 'una', 'pendiente', 'en', 'la', 'escala', 'ALSFRS-R', 'antes', 'del', 'estudio', '≥0,4', 'puntos', 'al', 'mes', '(', 'calculada', 'como', '[', 'puntuación', '48-Screening', 'ALSFRS-R]/tiempo', 'en', 'meses', 'desde', 'el', 'comienzo', 'de', 'los', 'síntomas', ')']


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [20]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [21]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [0, 516, 5167, 66, 34901, 4088, 49160, 8846, 507, 55, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [22]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [0, 516, 5167, 10632, 34901, 4088, 49160, 8846, 507, 326, 48179, 11032, 907, 18401, 275, 68, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [23]:
example = datasets["train"][2]
print(example["tokens"])

['Título', 'científico', ':', 'Estudio', 'de', 'fase', '1-3', 'para', 'evaluar', 'la', 'eficacia', ',', 'la', 'seguridad', ',', 'la', 'farmacocinética', 'y', 'la', 'farmacodinámica', 'de', 'ION363', 'administrado', 'por', 'vía', 'intratecal', 'en', 'pacientes', 'con', 'esclerosis', 'lateral', 'amiotrófica', 'con', 'mutaciones', 'del', 'gen', 'fusionado', 'en', 'sarcoma', '(', 'ELA-FUS', ')']


In [24]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', 'ĠTÃŃtulo', 'ĠcientÃŃfico', 'Ġ', ':', 'ĠEstudio', 'Ġde', 'Ġfase', 'Ġ1', '-', '3', 'Ġpara', 'Ġevaluar', 'Ġla', 'Ġeficacia', 'Ġ,', 'Ġla', 'Ġseguridad', 'Ġ,', 'Ġla', 'Ġfarma', 'co', 'cin', 'Ã©tica', 'Ġy', 'Ġla', 'Ġfarma', 'co', 'din', 'Ã¡mica', 'Ġde', 'ĠI', 'ON', '36', '3', 'Ġadministr', 'ado', 'Ġpor', 'ĠvÃŃa', 'Ġin', 'trate', 'cal', 'Ġen', 'Ġpacientes', 'Ġcon', 'Ġesclerosis', 'Ġlateral', 'Ġam', 'io', 'trÃ³', 'fica', 'Ġcon', 'Ġmutaciones', 'Ġdel', 'Ġgen', 'Ġfus', 'ionado', 'Ġen', 'Ġsar', 'com', 'a', 'Ġ(', 'ĠELA', '-', 'F', 'US', 'Ġ)', '</s>']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [25]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(42, 68)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [26]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 16, 17, 18, 19, 19, 19, 19, 20, 21, 21, 21, 21, 22, 22, 23, 24, 25, 25, 25, 26, 27, 28, 29, 30, 31, 31, 31, 31, 32, 33, 34, 35, 36, 36, 37, 38, 38, 38, 39, 40, 40, 40, 40, 41, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [27]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

68 68


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [28]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [29]:
# Definición del label map
label_map = {
    'O': 0,
    'B-CW': 1,
    'I-CW': 2
}

In [30]:
# Función para preprocesar las etiquetas
def preprocess_labels(examples):
    # Convertir las etiquetas usando el label_map
    return {
        f"{task}_tags": [
            [label_map[label] for label in labels] for labels in examples[f"{task}_tags"]
        ]
    }

In [31]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [32]:
datasets = datasets.map(preprocess_labels, batched=True)

# Tokenizar y alinear las etiquetas
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2308 [00:00<?, ? examples/s]

Map:   0%|          | 0/1714 [00:00<?, ? examples/s]

Map:   0%|          | 0/2097 [00:00<?, ? examples/s]

Map:   0%|          | 0/2308 [00:00<?, ? examples/s]

Map:   0%|          | 0/1714 [00:00<?, ? examples/s]

Map:   0%|          | 0/2097 [00:00<?, ? examples/s]

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [33]:
label_list = ['O', 'B-CW', 'I-CW']

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-large-bne and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [83]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy="epoch",  # Cambiado de evaluation_strategy a eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    #push_to_hub=True,
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=100
)


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [37]:
metric = load_metric("seqeval")

<ipython-input-37-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


This metric takes list of labels for the predictions and references:

In [38]:
labels = [label_list[label_map[i]] for i in example[f"{task}_tags"]]

# Calcular la métrica usando las etiquetas convertidas
metric.compute(predictions=[labels], references=[labels])

{'CW': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 9},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [39]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [84]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

We can now finetune our model by just calling the `train` method:

In [85]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.463689,0.630854,0.876788,0.733762,0.907791
2,No log,0.654768,0.512123,0.864986,0.643346,0.867320
3,No log,0.585770,0.536074,0.864986,0.661923,0.878677
4,0.007500,0.477940,0.619613,0.858727,0.719832,0.906484
5,0.007500,0.565052,0.604159,0.857296,0.708805,0.898912
6,0.007500,0.463785,0.691687,0.858548,0.766137,0.922934


TrainOutput(global_step=870, training_loss=0.006683652976463581, metrics={'train_runtime': 274.7636, 'train_samples_per_second': 251.998, 'train_steps_per_second': 15.832, 'total_flos': 1949565741151200.0, 'train_loss': 0.006683652976463581, 'epoch': 6.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [86]:
trainer.evaluate()

{'eval_loss': 0.46368876099586487,
 'eval_precision': 0.6308543489449305,
 'eval_recall': 0.876788268955651,
 'eval_f1': 0.7337623466028136,
 'eval_accuracy': 0.9077914926811361,
 'eval_runtime': 7.4802,
 'eval_samples_per_second': 229.137,
 'eval_steps_per_second': 14.438,
 'epoch': 6.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [88]:
predictions, labels, _ = trainer.predict(tokenized_datasets["dev"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CW': {'precision': 0.6308543489449305,
  'recall': 0.876788268955651,
  'f1': 0.7337623466028136,
  'number': 5592},
 'overall_precision': 0.6308543489449305,
 'overall_recall': 0.876788268955651,
 'overall_f1': 0.7337623466028136,
 'overall_accuracy': 0.9077914926811361}

In [89]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CW': {'precision': 0.5141444114737883,
  'recall': 0.8397415185783522,
  'f1': 0.6377914110429447,
  'number': 6190},
 'overall_precision': 0.5141444114737883,
 'overall_recall': 0.8397415185783522,
 'overall_f1': 0.6377914110429447,
 'overall_accuracy': 0.8869151688929295}

In [60]:
# Guardar modelo final
trainer.save_model("roberta-large-bne-finetuned-ner-seed-100")

In [61]:
!zip -r "./roberta-large-bne-finetuned-ner-seed-100.zip" "./roberta-large-bne-finetuned-ner-seed-100"

  adding: roberta-large-bne-finetuned-ner-seed-100/ (stored 0%)
  adding: roberta-large-bne-finetuned-ner-seed-100/tokenizer.json (deflated 72%)
  adding: roberta-large-bne-finetuned-ner-seed-100/tokenizer_config.json (deflated 77%)
  adding: roberta-large-bne-finetuned-ner-seed-100/config.json (deflated 50%)
  adding: roberta-large-bne-finetuned-ner-seed-100/vocab.json (deflated 60%)
  adding: roberta-large-bne-finetuned-ner-seed-100/merges.txt (deflated 57%)
  adding: roberta-large-bne-finetuned-ner-seed-100/special_tokens_map.json (deflated 85%)
  adding: roberta-large-bne-finetuned-ner-seed-100/training_args.bin (deflated 51%)
  adding: roberta-large-bne-finetuned-ner-seed-100/model.safetensors (deflated 11%)


In [62]:
!rm -fr "roberta-large-bne-finetuned-ner"

### Seed: 500

In [67]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy="epoch",  # Cambiado de evaluation_strategy a eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    #push_to_hub=True,
    save_total_limit=1, # Save only 1 checkpoint (the best performing model)
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=500
)


In [68]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

In [69]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.442964,0.757304,0.829757,0.791876,0.936149
2,No log,0.397518,0.644223,0.855508,0.734982,0.909237
3,No log,0.516710,0.616286,0.864807,0.719696,0.904969
4,0.007700,0.377332,0.765862,0.850501,0.805965,0.938306
5,0.007700,0.426519,0.716970,0.866595,0.784714,0.932088
6,0.007700,0.488741,0.679563,0.866953,0.761905,0.923003
7,0.006200,0.545848,0.629135,0.867310,0.729268,0.910063
8,0.006200,0.509805,0.719484,0.848534,0.778699,0.929450
9,0.006200,0.432838,0.658923,0.862303,0.747018,0.919997


TrainOutput(global_step=1305, training_loss=0.005778698217823131, metrics={'train_runtime': 416.6835, 'train_samples_per_second': 166.169, 'train_steps_per_second': 10.44, 'total_flos': 2943902313522120.0, 'train_loss': 0.005778698217823131, 'epoch': 9.0})

In [70]:
trainer.evaluate()

{'eval_loss': 0.37733152508735657,
 'eval_precision': 0.7658615136876007,
 'eval_recall': 0.8505007153075823,
 'eval_f1': 0.8059650906625997,
 'eval_accuracy': 0.9383058780342312,
 'eval_runtime': 7.4666,
 'eval_samples_per_second': 229.555,
 'eval_steps_per_second': 14.464,
 'epoch': 9.0}

In [71]:
predictions, labels, _ = trainer.predict(tokenized_datasets["dev"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CW': {'precision': 0.7658615136876007,
  'recall': 0.8505007153075823,
  'f1': 0.8059650906625997,
  'number': 5592},
 'overall_precision': 0.7658615136876007,
 'overall_recall': 0.8505007153075823,
 'overall_f1': 0.8059650906625997,
 'overall_accuracy': 0.9383058780342312}

In [72]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CW': {'precision': 0.7300796812749004,
  'recall': 0.8289176090468497,
  'f1': 0.7763655621122711,
  'number': 6190},
 'overall_precision': 0.7300796812749004,
 'overall_recall': 0.8289176090468497,
 'overall_f1': 0.7763655621122711,
 'overall_accuracy': 0.9423211413385552}

In [73]:
# Guardar modelo final
trainer.save_model("roberta-large-bne-finetuned-ner-seed-500")

In [74]:
!zip -r "./roberta-large-bne-finetuned-ner-seed-500.zip" "./roberta-large-bne-finetuned-ner-seed-500"

  adding: roberta-large-bne-finetuned-ner-seed-500/ (stored 0%)
  adding: roberta-large-bne-finetuned-ner-seed-500/tokenizer.json (deflated 72%)
  adding: roberta-large-bne-finetuned-ner-seed-500/tokenizer_config.json (deflated 77%)
  adding: roberta-large-bne-finetuned-ner-seed-500/config.json (deflated 50%)
  adding: roberta-large-bne-finetuned-ner-seed-500/vocab.json (deflated 60%)
  adding: roberta-large-bne-finetuned-ner-seed-500/merges.txt (deflated 57%)
  adding: roberta-large-bne-finetuned-ner-seed-500/special_tokens_map.json (deflated 85%)
  adding: roberta-large-bne-finetuned-ner-seed-500/training_args.bin (deflated 51%)
  adding: roberta-large-bne-finetuned-ner-seed-500/model.safetensors (deflated 11%)


### Seed: 1000

In [76]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy="epoch",  # Cambiado de evaluation_strategy a eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    #push_to_hub=True,
    save_total_limit=1, # Save only 1 checkpoint (the best performing model)
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=1000
)


In [77]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

In [78]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.592365,0.612581,0.877682,0.721552,0.905199
2,No log,0.424305,0.735825,0.837804,0.783510,0.930987
3,No log,0.461425,0.743382,0.853720,0.794739,0.935323
4,0.007100,0.443050,0.743748,0.861588,0.798343,0.935736
5,0.007100,0.410234,0.731914,0.866595,0.793581,0.932960
6,0.007100,0.439631,0.743036,0.849070,0.792522,0.934245
7,0.005300,0.477643,0.696071,0.852289,0.766300,0.923324
8,0.005300,0.475146,0.588739,0.860157,0.699026,0.895769
9,0.005300,0.671158,0.581910,0.865165,0.695815,0.892282
10,0.005300,0.521556,0.627092,0.864270,0.726822,0.908916


TrainOutput(global_step=1450, training_loss=0.005420943621931405, metrics={'train_runtime': 465.4691, 'train_samples_per_second': 148.753, 'train_steps_per_second': 9.345, 'total_flos': 3243875689041864.0, 'train_loss': 0.005420943621931405, 'epoch': 10.0})

In [79]:
trainer.evaluate()

{'eval_loss': 0.4102339744567871,
 'eval_precision': 0.7319136082162815,
 'eval_recall': 0.8665951359084406,
 'eval_f1': 0.7935806108245312,
 'eval_accuracy': 0.932960124810719,
 'eval_runtime': 7.7845,
 'eval_samples_per_second': 220.182,
 'eval_steps_per_second': 13.874,
 'epoch': 10.0}

In [81]:
predictions, labels, _ = trainer.predict(tokenized_datasets["dev"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CW': {'precision': 0.7319136082162815,
  'recall': 0.8665951359084406,
  'f1': 0.7935806108245312,
  'number': 5592},
 'overall_precision': 0.7319136082162815,
 'overall_recall': 0.8665951359084406,
 'overall_f1': 0.7935806108245312,
 'overall_accuracy': 0.932960124810719}

In [82]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CW': {'precision': 0.6906802356722014,
  'recall': 0.8332794830371567,
  'f1': 0.7553082442524527,
  'number': 6190},
 'overall_precision': 0.6906802356722014,
 'overall_recall': 0.8332794830371567,
 'overall_f1': 0.7553082442524527,
 'overall_accuracy': 0.932949856633331}